# The heat equation problem 

Is a PDE that depends also on the time (t); thus, to solve this equation we need to solve n-variational problem (1 problem for each t-step)

In [1]:
import matplotlib as mpl
import pyvista
import ufl
import numpy as np

from petsc4py import PETSc
from mpi4py import MPI

from dolfinx import fem, mesh, io, plot

# Define temporal parameters
t = 0  # Start time
T = 1.0  # Final time
num_steps = 50
dt = T / num_steps  # time step size

# Define mesh on a rectangular domain Ω [-2,2]x[-2,2], with triangular elements 
nx, ny = 50, 50
domain = mesh.create_rectangle(MPI.COMM_WORLD, [np.array([-2, -2]), np.array([2, 2])],[nx, ny], mesh.CellType.triangle)


V = fem.functionspace(domain, ("Lagrange", 1)) # define the vector space of continuous piecewise linear functions (Lagrange: 1)

define initial condition and boundary condition:

u_0= e^(-a*x^2 - a*y^2)

u_D=0 <---Dirichlet boundary condition

In [2]:
# Create initial condition
def initial_condition(x, a=5):
    return np.exp(-a * (x[0]**2 + x[1]**2))


# Create boundary condition
fdim = domain.topology.dim - 1  # dimension of the facets
boundary_facets = mesh.locate_entities_boundary(domain, fdim, lambda x: np.full(x.shape[1], True, dtype=bool))
# the function: lambda x: np.full(x.shape[1], True, dtype=bool)
# basically says: "Give me all the facets that are actually on the boundary of the domain, WITHOUT ANY ADDITIONAL SELECTION".


bc = fem.dirichletbc(PETSc.ScalarType(0), fem.locate_dofs_topological(V, fdim, boundary_facets), V) # this is how I define the bc


XDMFiles are very convenient for time-dependent problems because they can store mesh and solutions separately.

So, they will be able to save memory, for example by storing a single mesh and various solutions (relative to that single mesh).

In [3]:
xdmf=io.XDMFFile(domain.comm,'diffusion.xdmf','w') # I put the xdmf file (which I call diffusion.xdmf) in write mode ('w')
xdmf.write_mesh(domain)
# domain.comm ensures that we use as the only communicator: COMM_WORLD 

# Imposing the initial condition

Now I define the solution uh

I remember that there are 2 ways to impose the initial condition u0 to u_n (with n=0):

-L2_projection (u_n would be the L2_projection of u0)<---this would require finding the VARIATIONAL formula of u_n=u0

-Interpolation of u0 on the vector space V 


We decide to choose interpolation!!!

In [4]:
uh=fem.Function(V)
uh.name='uh'
uh.interpolate(initial_condition)

xdmf.write_function(uh,t) # let's save the time-step->0 in the xdmf file


# Let's define the u_n that will change at each time-step 
# At t=0 we have: u_n = initial condition !!!
u_n=fem.Function(V)
u_n.name='u_n'
u_n.interpolate(initial_condition)

# Define the variational equation 

In [5]:
# define the variational problem 
u,v=ufl.TrialFunction(V),ufl.TestFunction(V)
f=fem.Constant(domain,PETSc.ScalarType(0))

a= u*v*ufl.dx + dt* ufl.dot(ufl.grad(u),ufl.grad(v))*ufl.dx
L= (u_n + dt*f)*v*ufl.dx 

Remember that the problem in question is time-dependent!!

So I will have to solve a sequence of variational problems (a(u,v)=L_n+1(v))

So f and a new u_n (that is, u_n+1) will be calculated at each new time-step

In [6]:
bilinear_form=fem.form(a)
linear_form=fem.form(L)

Remember that, solving the variational formula translates into 

solving a linear problem of the type: Ax=b (where x will contain the values of u_n at timestep n+1)

Note then that, given the nature of the problem, the matrix A will always be the same for any time-step 

(so I decide to assemble it only once to save memory)

while b will change at each timestep, because b depends on L(v), which in turn also depends on u_n

and since, in fact, u_n changes at each time-step then b WILL ALSO CHANGE AT EACH TIME-STEP

In [7]:
from dolfinx.fem.petsc import assemble_vector, assemble_matrix, create_vector, apply_lifting, set_bc

A=assemble_matrix(bilinear_form,bcs=[bc])
A.assemble()
b=create_vector(linear_form) # unlike A, we CREATED b (and did not assemble it!)

Since we assembled A, we will no longer use the class: dolfinx.fem.petsc.LinearProblem

to solve the linear problem (as we did in previous problems)

But, we will create a linear algebra solver using PETSc

In [8]:
from petsc4py import PETSc

solver=PETSc.KSP().create(domain.comm)

solver.setOperators(A)

solver.setType(PETSc.KSP.Type.PREONLY)
solver.getPC().setType(PETSc.PC.Type.LU)

In [9]:
# pyvista.start_xvfb()

grid = pyvista.UnstructuredGrid(*plot.vtk_mesh(V))

plotter = pyvista.Plotter(window_size=(400,400))
plotter.open_gif("u_time.gif", fps=2)

grid.point_data["uh"] = uh.x.array # Associates a scalar field (i.e. the values of uh) to the grid nodes
warped = grid.warp_by_scalar("uh", factor=1) 

viridis = mpl.colormaps.get_cmap("viridis").resampled(25)
sargs = dict(title_font_size=25, label_font_size=20, fmt="%.2e", color="black",
             position_x=0.1, position_y=0.8, width=0.8, height=0.1)

renderer = plotter.add_mesh(warped, show_edges=True, lighting=False,
                            cmap=viridis, scalar_bar_args=sargs,
                            clim=[0, max(uh.x.array)])

In [ ]:
for i in range(num_steps):
    t += dt

    # Update the right hand side reusing the initial vector
    with b.localForm() as loc_b:
        loc_b.set(0)
    assemble_vector(b, linear_form)

    # Apply Dirichlet boundary condition to the vector
    apply_lifting(b, [bilinear_form], [[bc]])
    b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    set_bc(b, [bc])

    # Solve linear problem
    solver.solve(b, uh.x.petsc_vec)
    uh.x.scatter_forward()

    # Update solution at previous time step (u_n)
    u_n.x.array[:] = uh.x.array

    # Write solution to file
    xdmf.write_function(uh, t)
    # Update plot
    new_warped = grid.warp_by_scalar("uh", factor=1)
    warped.points[:, :] = new_warped.points
    warped.point_data["uh"][:] = uh.x.array
    plotter.write_frame()
plotter.close()
xdmf.close()